In [1]:
import os
os.chdir("../")
os.chdir("FinalDataset")

In [2]:
os.getcwd()

'/Volumes/Rohith/url_classification_dl/FinalDataset'

In [3]:
import pandas as pd
import tensorflow as ts
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [4]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

In [5]:
from sklearn.preprocessing import MinMaxScaler

In [6]:
m = ts.keras.metrics.CategoricalAccuracy()

In [7]:
from keras.wrappers.scikit_learn import KerasClassifier

from keras.utils import np_utils

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import KFold

In [13]:
data = pd.read_csv("feature.csv")

In [14]:
data.drop(columns='Unnamed: 0',inplace=True)
data.replace(True,1,inplace = True)
data.replace(False,0,inplace = True)
y = data["File"]

In [15]:
data = data.drop(columns = "File")

In [63]:
encoder = LabelEncoder()
encoder.fit(y)
Y = encoder.transform(y)

In [17]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(data)
X = pd.DataFrame(X)

In [18]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [49]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

input_dim = len(data.columns)
model = Sequential()
model.add(Dense(128, input_dim = input_dim , activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy',f1_m,precision_m, recall_m] )
    

In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [51]:
model.fit(X_train,np_utils.to_categorical(y_train),epochs = 24,validation_split=0.3, batch_size = 128)

Epoch 1/24
224/224 [==============================] - 2s 3ms/step - loss: 1.4561 - accuracy: 0.3635 - f1_m: 0.0994 - precision_m: 0.4068 - recall_m: 0.0616 - val_loss: 1.0443 - val_accuracy: 0.5627 - val_f1_m: 0.4907 - val_precision_m: 0.8005 - val_recall_m: 0.3547
Epoch 2/24
224/224 [==============================] - 0s 1ms/step - loss: 1.0182 - accuracy: 0.5767 - f1_m: 0.5264 - precision_m: 0.7863 - recall_m: 0.3982 - val_loss: 0.9311 - val_accuracy: 0.6079 - val_f1_m: 0.5999 - val_precision_m: 0.8124 - val_recall_m: 0.4763
Epoch 3/24
224/224 [==============================] - 0s 1ms/step - loss: 0.9267 - accuracy: 0.6184 - f1_m: 0.6089 - precision_m: 0.7961 - recall_m: 0.4939 - val_loss: 0.8830 - val_accuracy: 0.6206 - val_f1_m: 0.6341 - val_precision_m: 0.7953 - val_recall_m: 0.5279
Epoch 4/24
224/224 [==============================] - 0s 1ms/step - loss: 0.8716 - accuracy: 0.6450 - f1_m: 0.6376 - precision_m: 0.8069 - recall_m: 0.5280 - val_loss: 0.8534 - val_accuracy: 0.6726 - va

In [52]:
y_pred = model.predict(X_test)

In [53]:
predicted = np.argmax(y_pred, axis=1)

In [54]:
from sklearn.metrics import classification_report
print(accuracy_score(y_test,predicted))

0.7944693024279322


In [55]:
target_names = ['Benign_list_big_final','DefacementSitesURLFiltered','Malware_dataset','phishing_dataset','spam_dataset']
print(classification_report(y_test, predicted, target_names=target_names))

                            precision    recall  f1-score   support

     Benign_list_big_final       0.93      0.85      0.89      2735
DefacementSitesURLFiltered       0.72      0.70      0.71      2499
           Malware_dataset       0.74      0.88      0.80      2834
          phishing_dataset       0.75      0.69      0.72      2477
              spam_dataset       0.83      0.83      0.83      3088

                  accuracy                           0.79     13633
                 macro avg       0.80      0.79      0.79     13633
              weighted avg       0.80      0.79      0.79     13633



In [56]:
model.save("Model_v1")

INFO:tensorflow:Assets written to: Model_v1/assets
